In [114]:
import urllib2, re
import sys,os
from firebase import firebase
import hashlib
from datetime import datetime

In [115]:
src_url = 'http://www.calendarwiz.com/calendars/rssfeeder.xml?crd=olympiawa&len=500&days=100&events=200&title=City%20of%20Olympia%20Calendar&cat=70522'
req = urllib2.Request(src_url)
page = urllib2.urlopen(req).read()

detailLinks = []
# source html file
htmlFile = open('olympiagov.html','wb')
htmlFile.write(page)
htmlFile.close()

detailRE = re.compile(r'<link>(.*)</link>')
detail = detailRE.findall(page)

for i in range(2, len(detail)):
    detailLinks.append(detail[i].replace("&amp;","&"))

titles = []
times = []
dates = []
locations = []

for i in range(0, len(detailLinks)):
    detail_url = detailLinks[i]
    req_detail = urllib2.Request(detail_url)
    page_detail = urllib2.urlopen(req_detail).read()
    
    titleRE = re.compile(r'<p id="titletag">(.*)<p>')
    # [\s] to match any white space and line break
    timeRE = re.compile(r'</span>[\s]*(.*)[\s]*<br>\(Time Zone: US/Pacific\)')
    dateRE = re.compile(r'<p>[\s](.*)<span style=\'color:#ccc\'>')
    locationRE = re.compile(r'name="location0">(.*)</a>')
    
    titles.append(titleRE.findall(page_detail)[0])
    if len(timeRE.findall(page_detail)) > 0:
        times.append(timeRE.findall(page_detail)[0])
    else:
        times.append("NA")
    if len(dateRE.findall(page_detail)) > 0:
        date = datetime.strptime(dateRE.findall(page_detail)[0].replace("\t",""),'%A, %B %d, %Y ')
        dates.append(date)
    else:
        dates.append("NA")
    if len(locationRE.findall(page_detail)) > 0:
        locations.append(locationRE.findall(page_detail)[0])
    else:
        locations.append("NA")


# push date to firebase
f = firebase.FirebaseApplication('https://wa-state-calendar.firebaseio.com', None)

for i in range(len(titles)):
    agent = "Olympia City Council"
    title = titles[i]
    date = dates[i]
    time = times[i]
    location = locations[i]
    if len(time) > 10:
        time = "Flexible/On Appointment"

    # use MD5 string as index to avoid duplicates
    tmpString = agent+title+"Olympia"+date.strftime('%m/%d/%Y')+time+location;
    MD5String = hashlib.md5(tmpString).hexdigest()

    f.put('meeting/',MD5String,
        {
        "MD5index":MD5String,
         "title": title, 
         "date": date,
         "time": time,
         "location": location,
         "cancelOrNot":"No",
         "board": "NA",
         "city": "Olympia",
         "zipcode": 98501,
         "description": "NA",
         "contactName": "NA",
         "contactEmail": "NA",
         "contactPhone": "NA",
         "weblink": "NA",
         "summary": "NA"
    })

# store data in a file (just for debugging purpose)     
events = []
for i in range(len(titles)):
    item = 'Olympia City Council '+titles[i]+' '+locations[i]
    events.append(item)
events.append(datetime.now().strftime("%I:%M %p on %B %d, %Y"))

webFile = open('crawler_olympiagov.html','wb')
for event in events:
    webFile.write(event)
    webFile.write('\n')
webFile.close()

